#### CSCE 670 :: Information Storage and Retrieval :: Texas A&M University :: Spring 2019


# Homework 1:  Modeling Text + Link Analysis + SEO

### 100 points [6% of your final grade]

### Due: Monday, February 8, 2019 by 11:59pm

*Goals of this homework:* Understand the vector space model (TF-IDF, cosine) + BM25 works in searching. Explore real-world challenges of building a graph (in this case, from Epinions), implement and test the classic HITS algorithm over this graph. Experiment with real-world search engine optimization techniques.

*Submission instructions (eCampus):* To submit your homework, rename this notebook as `UIN_hw1.ipynb`. For example, my homework submission would be something like `555001234_hw1.ipynb`. Submit this notebook via eCampus (look for the homework 1 assignment there). Your notebook should be completely self-contained, with the results visible in the notebook. We should not have to run any code from the command line, nor should we have to run your code within the notebook (though we reserve the right to do so). So please run all the cells for us, and then submit.

*Late submission policy:* For this homework, you may use as many late days as you like (up to the 5 total allotted to you).

*Collaboration policy:* You are expected to complete each homework independently. Your solution should be written by you without the direct aid or help of anyone else. However, we believe that collaboration and team work are important for facilitating learning, so we encourage you to discuss problems and general problem approaches (but not actual solutions) with your classmates. You may post on Piazza, search StackOverflow, etc. But if you do get help in this way, you must inform us by **filling out the Collaboration Declarations at the bottom of this notebook**. 

*Example: I found helpful code on stackoverflow at https://stackoverflow.com/questions/11764539/writing-fizzbuzz that helped me solve Problem 2.*

The basic rule is that no student should explicitly share a solution with another student (and thereby circumvent the basic learning process), but it is okay to share general approaches, directions, and so on. If you feel like you have an issue that needs clarification, feel free to contact either me or the TA.

# Part 1: Modeling Text (50 points)

### TF-IDF


First, you will need to download the review.json file from the Resources tab on Piazza, a collection of about 7,000 Yelp reviews we sampled from the [Yelp Dataset Challenge](https://www.yelp.com/dataset_challenge). You'll see that each line corresponds to a review on a particular business. Each review has a unique "ID" and the text content is in the "review" field. You need to load the json file first. We already have done some basic preprocessing on the reviews, so you can just tokenize each review using whitespace.

Here you can treat each review as a document. Given a query, you need to calculate its TF-IDF score in each review.  For this homework, we will define the TF-IDF as follows:

`TF = number of times word occurs in a document`

`IDF = log(total number of documents / number of documents containing the word)`

### A) Ranking with simple sums of TF-IDF scores

To start out with, for a multi-word query, we will rank documents by a simple sum of the TF-IDF scores for the query terms in the document. 

Please calculate this TF-IDF sum score for queries `"best bbq"` and `"kid fun and food"`. You need to report the Top-10 reviews with highest TF-IDF scores for each query. Your output should look like this:

Query "best bbq"

Rank Review_ID score

1 dhskfhjskfhs 0.55555

...



Query "kid fun and food"

Rank Review_ID score

1 dhskfhjskfhs 0.55555

...

In [49]:
# Your code here
import numpy as np
import json
from operator import itemgetter
import math

documents = []
with open('review.json') as f:
    for line in f:
        documents.append(json.loads(line))


# calculate the idf weight for these terms in each query
queryOne = ["best", "bbq"]
queryTwo = ["kid", "fun", "and", "food"]

# calculate number of documents in contain that term
idf_queryOne_document = []
idf_queryTwo_document = []

def idf(term, documents):
    size = len(documents)
    occurance = 0
    for document in documents:
        review = document.get("review")
        review = review.split()
        if term in review:
            occurance += 1
    return math.log10(size / occurance)


for term in queryOne:
    idf_queryOne_document.append(idf(term, documents))
    print(term, idf(term, documents))
    
    
for term in queryTwo:
    idf_queryTwo_document.append(idf(term, documents))
    print(term, idf(term, documents))
    

    
# calculate the TF for each review, and then get the TF-IDF score


for document in documents:
    review = document.get("review")
    review = review.split()
    tempTF_IDF = 0;
    for i in range(len(queryOne)):
        term = queryOne[i]
        TF = review.count(term)
        if document.get("id") == "YbQvHNrjZJ38mnh5rLuq7w":
            print(term, TF)
        tempTF_IDF += (TF * idf_queryOne_document[i])  #math.pow(TF * IDF_queryOne[i], 2)
    document["TF-IDF-One"] = tempTF_IDF #math.sqrt(tempTF_IDF)  #tempTF_IDF
    
for document in documents:
    review = document.get("review")
    review = review.split()
    tempTF_IDF = 0;
    for i in range(len(queryTwo)):
        term = queryTwo[i]
        TF = review.count(term)
        if document.get("id") == "YbQvHNrjZJ38mnh5rLuq7w":
            print(term, TF)
        tempTF_IDF += (TF * idf_queryTwo_document[i])  #math.pow(TF * IDF_queryTwo[i], 2)
    document["TF-IDF-Two"] = tempTF_IDF#math.sqrt(tempTF_IDF)
    




best 0.8511875910514064
bbq 1.9050888219269386
kid 1.8898488553702018
fun 1.3017382071174817
and 0.05194128559950884
food 0.561258378180342
best 0
bbq 6
kid 0
fun 0
and 7
food 3
actually i d put it at 4 1 2 stars not perfect but prety good food can t say i m an alice cooper fan but we did see this on man vs food and the opportunity came up to go there we stopped in mid week for lunch not crowded the bar tender was great attentive personable good recommendations on food but not intrusive my wife had the lady gaga sausage appetizer fantastic sausage spicy great sauce served with celery stalks and bleu cheese optional she likes a good sausage let s leave it at that and alice cooper delivered since the bartender claimed the place is a bbq house i went with a bbq beef sandwich with chili and fries for the sides while the pig in me would have wanted more of everything it was the perfect size for lunch the brisket was done perfectly we travel with a smoker so i know a little about bbq the bbq

In [40]:
# Show us the result for "best bbq"
sortedDocument = sorted(documents, key=itemgetter('TF-IDF-One'), reverse=True)
print('Query "best bbq"')
print('Rank  Review_ID              score')
# Rank Review_ID score
for i in range(10):
    ID = sortedDocument[i].get("id")
    score_one = sortedDocument[i].get("TF-IDF-One")
    print(i, "   ", ID, score_one)

Query "best bbq"
Rank  Review_ID              score
0     YbQvHNrjZJ38mnh5rLuq7w 11.430532931561633
1     P31kXP4oan6ZQm69TN6tIA 9.525444109634693
2     x5esEK6J9XkA_vbvVbG8Gg 8.471542878759161
3     mWs26TrBM7ogwCM9UfVJFg 7.6203552877077545
4     NCfX4AxDvQ3QRyXKtmhVwQ 7.6203552877077545
5     e5INq6DAZn2zMHicKQl07Q 6.566454056832223
6     4WTG1-9mw8YHEyaTu8dQww 6.566454056832223
7     x3n_l3GhBx78y6jWX4fStg 5.958313137359845
8     Wp8jYXL1DQrgrnZIFmufFg 5.715266465780816
9     jrEx93eYKIjCW2nrkwjZpQ 5.715266465780816


In [14]:
# Show us the result for "kid fun and food"
sortedData = sorted(documents, key=itemgetter('TF-IDF-Two'), reverse=True)
print('Query "kid fun and food"')
print('Rank  Review_ID              score')
# Rank Review_ID score
for i in range(10):
    ID = sortedData[i].get("id")
    score_two = sortedData[i].get("TF-IDF-Two")
    print(i, "   ", ID, score_two)


Query "kid fun and food"
Rank  Review_ID              score
0     7o_hciiXEMNQkXfVl0F0XQ 9.641872501183393
1     JKLUXUovJCU6kbcdin74NQ 8.692007941255747
2     IA8TOfGKI-Il-70BsB6HgA 8.132369151667877
3     Kytq1NbFIDDCXUculSqT8g 7.2912649956941475
4     MF6rPRx9jz-g8S5P_ZIdyg 7.080045177182006
5     bjoedmJ4_DZP5JnfXVaC-w 6.825484846867428
6     I00B-QG5uTKvwCK7x9ejeA 6.8021590858855445
7     BVGRJgDJGEhSfgIPCan7vQ 6.721602275668411
8     wMB3cI3-xhxM_BpmppY9RQ 6.425196423168579
9     vTGDEQGp6EPlwdMJUnTb7A 6.0414680741761755


### B) Ranking with TF-IDF + Cosine

Instead of using the sum of TF-IDF scores, let's try the classic cosine approach for ranking. You should still use the TF-IDF scores to weigh each term, but now use the cosine between the query vector and the document vector to assign a similarity score. You can try the same two queries as before and report the results. (Top-10 reviews which are similar to the query)

In [35]:
# Your code here

def idf_query(term, documents):
    size = len(documents)
    occurance = 0
    for document in documents:
        if term in document:
            occurance += 1
    return math.log10(size / occurance)       

    
    
queries = [queryOne, queryTwo]
idf_queryOne_query = []
idf_queryTwo_query = []


for term in queryOne:
    idf_queryOne_query.append(idf_query(term, queries))

for term in queryTwo:
    idf_queryTwo_query.append(idf_query(term, queries))
    
def tf_idf_score(idf_query, document, term, i):
    tf = document.count(term)
    idf = idf_query[i]
    return tf * idf

k = 0
for document in documents:
    review = document.get("review").split()
    #  cosine = a / b * c ---> a = sum of tf_idf_query * tf_idf_document  b = math.sqrt(sum of tf_idf_query ** 2)
    #                          c = math.sqrt(sum of tf_idf_document ** 2)
    a = 0
    b = 0
    c = 0
    for i in range(len(queryOne)):
        term = queryOne[i]
        tf_idf_query = tf_idf_score(idf_queryOne_query, queryOne, term, i)
        tf_idf_document = tf_idf_score(idf_queryOne_document, review, term, i)
        a += (tf_idf_query * tf_idf_document)
        b += (tf_idf_query ** 2)
        c += (tf_idf_document ** 2)
        print(term, review)
        k += 1
    document["cosine_one"] = (a / (math.sqrt(b) * math.sqrt(c)))


for document in documents:
    review = document.get("review").split()
    #  cosine = a / b * c ---> a = sum of tf_idf_query * tf_idf_document  b = math.sqrt(sum of tf_idf_query ** 2)
    #                          c = math.sqrt(sum of tf_idf_document ** 2)
    a = 0
    b = 0
    c = 0
    for i in range(len(queryTwo)):
        term = queryTwo[i]
        tf_idf_query = tf_idf_score(idf_queryTwo_query, queryTwo, term, i)
        tf_idf_document = tf_idf_score(idf_queryTwo_document, review, term, i)
        a += (tf_idf_query * tf_idf_document)
        b += (tf_idf_query ** 2)
        c += (tf_idf_document ** 2)
    document["cosine_two"] = a / (math.sqrt(b) * math.sqrt(c))
    

    




    
    




best ['every', 'time', 'i', 'go', 'horrible', 'stale', 'taco', 'shells', 'hard', 'uncooked', 'beans', 'need', 'i', 'go', 'on', 'stay', 'clear', 'of', 'this', 'taco', 'bell']
bbq ['every', 'time', 'i', 'go', 'horrible', 'stale', 'taco', 'shells', 'hard', 'uncooked', 'beans', 'need', 'i', 'go', 'on', 'stay', 'clear', 'of', 'this', 'taco', 'bell']


ZeroDivisionError: float division by zero

In [28]:
# Show us the result for "best bbq"
sortedData = sorted(documents, key=itemgetter('cosine_one'), reverse=True)
print('Query "best bbq"')
print('Rank  Review_ID              score')
# Rank Review_ID score
for i in range(10):
    ID = sortedData[i].get("id")
    score_two = sortedData[i].get("cosine_one")
    print(i, "   ", ID, score_two)



Query "best bbq"
Rank  Review_ID              score
0     YbQvHNrjZJ38mnh5rLuq7w 18.985704242444896
1     P31kXP4oan6ZQm69TN6tIA 15.82142020203741
2     x5esEK6J9XkA_vbvVbG8Gg 14.070928147996511
3     mWs26TrBM7ogwCM9UfVJFg 12.657136161629928
4     NCfX4AxDvQ3QRyXKtmhVwQ 12.657136161629928
5     e5INq6DAZn2zMHicKQl07Q 10.90664410758903
6     4WTG1-9mw8YHEyaTu8dQww 10.90664410758903
7     x3n_l3GhBx78y6jWX4fStg 9.896543904566064
8     Wp8jYXL1DQrgrnZIFmufFg 9.492852121222448
9     jrEx93eYKIjCW2nrkwjZpQ 9.492852121222448


In [19]:
# Show us the result for "kid fun and food"
sortedData = sorted(documents, key=itemgetter('cosine_two'), reverse=True)
print('Query "kid fun and food"')
print('Rank  Review_ID              score')
# Rank Review_ID score
for i in range(10):
    ID = sortedData[i].get("id")
    score_two = sortedData[i].get("cosine_two")
    print(i, "   ", ID, score_two)



Query "kid fun and food"
Rank  Review_ID              score
0     7o_hciiXEMNQkXfVl0F0XQ 8.00740178725075
1     JKLUXUovJCU6kbcdin74NQ 7.218556345260381
2     IA8TOfGKI-Il-70BsB6HgA 6.753786390730205
3     Kytq1NbFIDDCXUculSqT8g 6.055264509116293
4     MF6rPRx9jz-g8S5P_ZIdyg 5.879850246788169
5     bjoedmJ4_DZP5JnfXVaC-w 5.668442468509218
6     I00B-QG5uTKvwCK7x9ejeA 5.649070843324132
7     BVGRJgDJGEhSfgIPCan7vQ 5.58216986055043
8     wMB3cI3-xhxM_BpmppY9RQ 5.336010128323373
9     vTGDEQGp6EPlwdMJUnTb7A 5.017330632492722


### C) Ranking with BM25

Finally, let's try the BM25 approach for ranking. Refer to [https://en.wikipedia.org/wiki/Okapi_BM25](https://en.wikipedia.org/wiki/Okapi_BM25) for the specific formula. You should choose k_1 = 1.2 and b = 0.75. You need to report the Top-10 reviews with highest BM25 scores for each query.


In [ ]:
# Your code here



In [ ]:
# Show us the result for "best bbq"



In [ ]:
# Show us the result for "kid fun and food"



Briefly discuss the differences you see between the three methods. Is there one you prefer? 

*add your discussion here*

# Part 2: Link Analysis (40 points)

## A Trust Graph


In this part, we're going to adapt the classic HITS approach to allow us to find not the most authoritative web pages, but rather to find the most trustworthy users. [Epinions.com](https://snap.stanford.edu/data/soc-Epinions1.html) is a general consumer review site with a who-trust-whom online social network. Members of the site can decide whether to ''trust'' each other. All the trust relationships interact and form the Web of Trust which is then combined with review ratings to determine which reviews are shown to the user. (Refer to: Richardson, Matthew, Rakesh Agrawal, and Pedro Domingos. "Trust management for the semantic web." International semantic Web conference. Springer, Berlin, Heidelberg, 2003.)

So, instead of viewing the world as web pages with hyperlinks (where pages = nodes, hyperlinks = edges), we're going to construct a graph of Epinions users and their "trust" on other users (so user = node, trust behavior = edge). Over this Epinions-user graph, we can apply the HITS approach to order the users by their hub-ness and their authority-ness.

You need to download the *Epinions_trust.txt* file from the Resources tab on Piazza. Each line represents the trust relationship between two users. Here is a toy example. Suppose you are given the following four lines:

* diane trust bob
* charlie trust bob 
* charlie trust alice
* bob trust charlie

The "trust" between each user pair denotes a directed edge between two nodes. E.g., the "diane" node has a directed edge to the "bob" node (as indicated by the first line). 

You should build a graph by parsing the data in the file we provide called *Epinions_trust.txt*. (Note: The edges are binary and directed.)

**Notes:**

* The edges are binary and directed.
* User can't trust himself/herself.
* Later you will need to implement the HITS algorithm on the graph you build here.

In [ ]:
# Here define your function for building the graph 
# by parsing the input file 
# Insert as many cells as you want


Please show us the size of the graph, i.e., the number of nodes and edges


In [ ]:
# Call your function to print out the size of the graph
# How you maintain the graph is totally up to you


## HITS Implementation

Your program will return the top 10 users with highest hub and authority scores. The **output** should be like:

Hub Scores

* user1 - score1
* user2 - score2
* ...
* user10 - score10

Authority Scores

* user1 - score1
* user2 - score2
* ...
* user10 - score10

You should follow these **rules**:

* Assume all nodes start out with equal scores.
* It is up to you to decide when to terminate the HITS calculation.
* There are HITS implementations out there on the web. However, remember, your code should be **your own**.


**Hints**:
* If you're using the matrix style approach, you should use [numpy.matrix](https://docs.scipy.org/doc/numpy/reference/generated/numpy.matrix.html).
* Scipy is built on top of Numpy and has support for sparse matrices. You most likely will not need to use Scipy unless you'd like to try out their sparse matrices.
* If you choose to use Numpy (and Scipy), please make sure your Anaconda environment include their latest versions.
* Test your parsing and HITS calculations using a handful of trust relationships, before moving on to the entire file we provide.
* We will evaluate the user ranks you provide as well as the quality of your code. So make sure that your code is clear and readable.

# Part 3: Search Engine Optimization (10 + 5 points)

For this part, your goal is to put on your "[search engine optimization](https://en.wikipedia.org/wiki/Search_engine_optimization)" hat. Your job is to create a webpage that scores highest for the query: **sajfd hfafbjhd** --- two terms, lower case, no quote. As of today (Jan 24, 2019), there are no hits for this query on either Google or Bing. Based on our discussions of search engine ranking algorithms, you know that several factors may impact a page's rank. Your goal is to use this knowledge to promote your own page to the top of the list.

What we're doing here is a form of [SEO contest](https://en.wikipedia.org/wiki/SEO_contest). While you have great latitude in how you approach this problem, you are not allowed to engage in any unethical or illegal behavior. Please read the discussion of "white hat" versus "black hat" SEO over at [Wikipedia](https://en.wikipedia.org/wiki/Search_engine_optimization).


**Rules of the game:**

* Somewhere in the page (possibly in the non-viewable source html) you must include your name or some other way for us to identify you.
* Your target page may only be a TAMU student page, a page on your own webserver, a page on a standard blog platform (e.g., wordpress), or some other primarily user-controlled page
* Your target page CAN NOT be a twitter account, a facebook page, a Yahoo Answers or similar page
* No wikipedia vandalism
* No yahoo/wiki answers questions
* No comment spamming of blogs
* If you have concerns/questions/clarifications, please post on Piazza and we will discuss

For your homework turnin for this part, you should provide us the URL of your target page and a brief discussion (2-4 paragraphs) of the strategies you are using. We will issue the query and check the rankings at some undetermined time in the next couple of weeks. You might guess that major search engines take some time to discover and integrate new pages: if I were you, I'd get a target page up immediately.

**Grading:**

* 5 points for providing a valid URL
* 5 points for a well-reasoned discussion of your strategy

** Bonus: **
* 1 point for your page appearing in the top-20 on Google or Bing
* 1 more point for your page appearing in the top-10 on Google or Bing
* 1 more point for your page appearing in the top-5 on Google or Bing
* 2 more points for your page being ranked first by Google or Bing. And, a vigorous announcement in class, and a high-five for having the top result!

What's the URL of your page?

*ADD YOUR INPUT HERE*

What's your strategy? (2-4 paragraphs)

*ADD YOUR INPUT HERE*

## Collaboration declarations

*If you collaborated with anyone (see Collaboration policy at the top of this homework), you can put your collaboration declarations here.*